In [1]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import torchmetrics
import pandas as pd

In [2]:
vbm_deep1 = os.path.join('output', 'GIN', 'scenario1', 'Reports-Deformable', "type-SVBP_fpr0001.nii.gz")
vbm_deep2 = os.path.join('output', 'GIN', 'scenario2', 'Reports-Deformable', "type-SVBP_fpr0001.nii.gz")
vbm_deep3 = os.path.join('output', 'GIN', 'scenario3', 'Reports-Deformable', "type-SVBP_fpr0001.nii.gz")
vbm_ants = os.path.join('output', 'GIN', 'ANTS', 'Reports-Deformable', "type-SVBP_fpr0001.nii.gz")
vbm_truth = os.path.join('output', 'GIN', 'ANTS-Mean', 'Reports-Deformable', "type-SVBP_fpr0001.nii.gz")

In [3]:
def get_normalized(vbm):
    return ((vbm - np.min(vbm)) / ( np.max(vbm) - np.min(vbm)) - 0.5 ) * 2

In [4]:
def get_labels(vbm, limit):
    x = np.where(vbm > limit)
    y = np.where(vbm < -limit)
    vbm[x] = 5
    vbm[y] = 4
    #print(len(np.where(vbm==4)[0]))
    #print(len(np.where(vbm==5)[0]))
    return vbm

In [6]:
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=6)

limit = 0.5
print(limit)
vbm_truth_nib = nib.load(vbm_truth).get_fdata()
vbm_truth_nib = get_normalized(vbm_truth_nib)
vbm_truth_labels = get_labels(vbm_truth_nib, limit)
vbm_truth_torch = torch.from_numpy(vbm_truth_labels.ravel()).type('torch.LongTensor')
print('-'*10)
for vbm in [vbm_ants, vbm_deep1, vbm_deep2, vbm_deep3]:
    vbm_nib = nib.load(vbm).get_fdata()    
    vbm_nib = get_normalized(vbm_nib)
    vbm_labels = get_labels(vbm_nib, limit)
    vbm_torch = torch.from_numpy(vbm_labels.ravel()).type('torch.LongTensor')
    dices = dicemetric(vbm_torch,vbm_truth_torch)
    print(dices)
    for i in range(4,6):
        a = np.where(vbm_torch == i)
        b = np.where(vbm_truth_torch == i)
        c = len(np.intersect1d(a,b)) / len(b[0]) * 100
        print(c)
    print('-'*10)

0.5
----------
tensor([0.9976,    nan,    nan,    nan, 0.1410, 0.1033])
12.88249694002448
12.31723526805494
----------
tensor([0.9953,    nan,    nan,    nan, 0.0914, 0.1784])
10.373317013463893
61.852015950376604
----------
tensor([0.9977,    nan,    nan,    nan, 0.0491, 0.1088])
4.467564259485924
10.899424014178113
----------
tensor([0.9949,    nan,    nan,    nan, 0.1297, 0.1440])
17.74785801713586
50.73105892778024
----------
